Demo of a single and 2 layer perceptron using numpy and pysyft

Based on http://iamtrask.github.io/2015/07/12/basic-python-network/

In [1]:
import numpy as np
from syft.syft import FloatTensor,SyftController

Using numpy

In [2]:
#4 inupts samples
x = np.array([ [0,0,1],[1,1,1],[1,0,1],[0,1,1] ])#works
#x = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])#xor gate need hidden layer
y = np.array([[0,1,1,0]]).T
#perceprton weights
w_initial = 2*np.random.random((len(x[0]),len(y[0]))) - 1
#activation function
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)#the derivative is sigmoid(sigmoid(x),1)
    return 1/(1+np.exp(-x))
#predicted y pre training
print(sigmoid(np.dot(x,w_initial)))
#training
w = w_initial.copy()
for j in range(60000):
    l1 = sigmoid(np.dot(x,w))
    l1_delta = (y - l1)*sigmoid(l1,1)
    w += x.T.dot(l1_delta)
    if j%10000 == 0:
        print("Error: ",np.average(np.abs((y - l1))))
#predicted y post training
print( l1 )

[[ 0.36532326]
 [ 0.0927639 ]
 [ 0.20277521]
 [ 0.18791128]]
Error:  0.564423858886
Error:  0.00795698840456
Error:  0.00559856961191
Error:  0.00456127190707
Error:  0.00394508271944
Error:  0.0035254944208
[[ 0.00390283]
 [ 0.99681523]
 [ 0.99740412]
 [ 0.0031816 ]]


Using pysyft

In [3]:
# connect to unity
identity = 'worker-0'
sc = SyftController(identity)

put data into openmined

In [25]:
X = sc.FloatTensor( x )
Y = sc.FloatTensor( y )
W = sc.FloatTensor( w_initial )
helper = sc.FloatTensor(np.array([[0,0,0,0]]).T)#needed as we don't have mm yet

FloatTensor.__init__: 12
FloatTensor.__init__: 13
FloatTensor.__init__: 14
FloatTensor.__init__: 15


In [5]:
#X.gpu()
#Y.gpu()
#W.gpu()

In [ ]:
Functions needed:
    - sub
    - mul
    - sigmoid_
    - average

In [7]:
#predicted pre training
print(X.mm(W).sigmoid())
#training
for j in range(10):
    l1 = helper.addmm(x,w)
    l1.sigmoid_()
    l1_delta = (y + l1.neg())*(l1*(1-l1.neg()))
    w.addmm_(x.T,l1_delta)
    if j%1 == 0:
        print("Error: ",(y + l1.neg()).abs().average())
#predicted y post training
print( l1 )

-0.9997712,	
-0.5591223,	
-0.1657273,	
0.2749218,	
